# TODO

- did use the API becuase you must enter in the version of uniprot ID, some canonical ID with no number were not found. which is why i use dbNSFP and search for the positions that match index of found canonical ukb id position

In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv

In [2]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

In [3]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/")
print(os.listdir())

['FTP_GRCh38_ensemblv92', '.DS_Store', 'K64_CADD38_coordinates.xlsx', 'NOTES.txt', 'UNIPROT_bed', 'GRCh37vsGRCh38_ensembl.sh', 'headerWhole.tsv', 'HUMAN_9606_uniprot_IDmapping', 'entriesSimpleUKBccds.txt', 'FTP_GRCh38_ensemblv97', '37_75vs97_ensembl', 'CYS_LYS_2012_to2018UKBpositions', 'Ensembl75', '38_92vs97_ensembl', 'FTP_GRCh37_ensemblv97', 'K64_CADD38_coordinates.txt', 'BIOMART_ENSEMBL']


## from API website testing, notes from CoordinatesP06733_K64.txt

```bash
Input: P06733_K64

Request URL: https://www.ebi.ac.uk/proteins/api/coordinates/location/P06733:64

# output
{
  "locations": [
    {
      "accession": "P06733",
      "taxid": 9606,
      "chromosome": "1",
      "ensemblTranslationId": "ENSP00000234590",
      "proteinStart": 64,
      "geneStart": 8870502,
      "proteinEnd": 64,
      "geneEnd": 8870500
    },
    {
      "accession": "P06733",
      "taxid": 9606,
      "chromosome": "1",
      "ensemblTranslationId": "ENSP00000493526",
      "proteinStart": 64,
      "geneStart": 8870502,
      "proteinEnd": 64,
      "geneEnd": 8870500
    }
  ]
}

# python
import requests, sys

requestURL = "https://www.ebi.ac.uk/proteins/api/coordinates/location/P06733:64"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.text
print(responseBody)
```

# DOES the uniprot API return bed file postions (0-indexed) or does it return 1-index?  
## this matters for mapping to CADD, since CADD file is 1-index I believe, i may need to convert coordinates to 0 index


## Experiments:

- CADD file chr1: 10001 ref T alt A, matches UCSC browser display
- checking if K64 position in CADD file matches K>__

```bash 

zcat chr1_GRCh38_inclannotationsCADD.tsv.gz | awk '$2 == 8870502 || $2 == 8870501 || $2 == 8870500' > K64_allPositions.txt

```

## Conclusion:
- CADD file is 0-based and matches the UKB API positions

** see file below **

In [4]:
K64 = pd.read_csv("K64_CADD38_coordinates.txt",sep='\t')
K64

,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,GC,CpG,motifECount,motifEName,motifEHIPos,motifEScoreChng,oAA,nAA,GeneID,FeatureID,GeneName,CCDS,Intron,Exon,cDNApos,relcDNApos,CDSpos,relCDSpos,protPos,relProtPos,Domain,Dst2Splice,Dst2SplType,minDistTSS,minDistTSE,SIFTcat,SIFTval,PolyPhenCat,PolyPhenVal,priPhCons,mamPhCons,verPhCons,priPhyloP,mamPhyloP,verPhyloP,bStatistic,targetScan,mirSVR-Score,mirSVR-E,mirSVR-Aln,cHmm_E1,cHmm_E2,cHmm_E3,cHmm_E4,cHmm_E5,cHmm_E6,cHmm_E7,cHmm_E8,cHmm_E9,cHmm_E10,cHmm_E11,cHmm_E12,cHmm_E13,cHmm_E14,cHmm_E15,cHmm_E16,cHmm_E17,cHmm_E18,cHmm_E19,cHmm_E20,cHmm_E21,cHmm_E22,cHmm_E23,cHmm_E24,cHmm_E25,GerpRS,GerpRSpval,GerpN,GerpS,tOverlapMotifs,motifDist,EncodeH3K4me1-sum,EncodeH3K4me1-max,EncodeH3K4me2-sum,EncodeH3K4me2-max,EncodeH3K4me3-sum,EncodeH3K4me3-max,EncodeH3K9ac-sum,EncodeH3K9ac-max,EncodeH3K9me3-sum,EncodeH3K9me3-max,EncodeH3K27ac-sum,EncodeH3K27ac-max,EncodeH3K27me3-sum,EncodeH3K27me3-max,EncodeH3K36me3-sum,EncodeH3K36me3-max,EncodeH3K79me2-sum,EncodeH3K79me2-max,EncodeH4K20me1-sum,EncodeH4K20me1-max,EncodeH2AFZ-sum,EncodeH2AFZ-max,EncodeDNase-sum,EncodeDNase-max,EncodetotalRNA-sum,EncodetotalRNA-max,Grantham,Dist2Mutation,Freq100bp,Rare100bp,Sngl100bp,Freq1000bp,Rare1000bp,Sngl1000bp,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCRawScore,PHRED
1,8870500,C,A,SNV,0,CodingTranscript,NON_SYNONYMOUS,7,missense,0.417219,0.013333,NaN,NaN,NaN,NaN,K,N,ENSG00000074800,ENST00000646539,ENO1,CCDS97.1,NaN,4/12,872.0,0.371855,192.0,0.147126,64.0,0.147465,ndomain,NaN,NaN,798,328,tolerated,0.10,benign,0.043,0.998,1,1,0.418,2.907,3.021,699,NaN,NaN,NaN,NaN,2,0,0,0,0,0,1,17,11,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,684.84,3.272140e-56,19.8,11.2,NaN,NaN,51.9226,7.94278,26.8278,5.17973,14.1775,3.46316,19.4358,5.10922,7.68046,2.38594,39.0690,8.53183,5.17637,1.23906,70.5810,11.8499,91.4021,17.2031,61.2727,11.0441,4.56415,0.99526,0.596333,0.100014,656.417,245.501,94.0,35,0,2,10,2,7,151,16,60,1450,NaN,NaN,NaN,7,7,3.057195,23.1
1,8870500,C,A,SNV,0,Intergenic,UPSTREAM,1,upstream,0.417219,0.013333,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000274258,ENST00000618792,MIR6728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798,328,NaN,NaN,NaN,NaN,0.998,1,1,0.418,2.907,3.021,699,NaN,NaN,NaN,NaN,2,0,0,0,0,0,1,17,11,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,684.84,3.272140e-56,19.8,11.2,NaN,NaN,51.9226,7.94278,26.8278,5.17973,14.1775,3.46316,19.4358,5.10922,7.68046,2.38594,39.0690,8.53183,5.17637,1.23906,70.5810,11.8499,91.4021,17.2031,61.2727,11.0441,4.56415,0.99526,0.596333,0.100014,656.417,245.501,NaN,35,0,2,10,2,7,151,16,60,1450,NaN,NaN,NaN,7,7,3.057195,23.1
1,8870500,C,G,SNV,0,Intergenic,UPSTREAM,1,upstream,0.417219,0.013333,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000274258,ENST00000618792,MIR6728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798,328,NaN,NaN,NaN,NaN,0.998,1,1,0.418,2.907,3.021,699,NaN,NaN,NaN,NaN,2,0,0,0,0,0,1,17,11,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,684.84,3.272140e-56,19.8,11.2,NaN,NaN,51.9226,7.94278,26.8278,5.17973,14.1775,3.46316,19.4358,5.10922,7.68046,2.38594,39.0690,8.53183,5.17637,1.23906,70.5810,11.8499,91.4021,17.2031,61.2727,11.0441,4.56415,0.99526,0.596333,0.100014,656.417,245.501,NaN,35,0,2,10,2,7,151,16,60,1450,NaN,NaN,NaN,7,7,3.083464,23.2
1,8870500,C,G,SNV,0,CodingTranscript,NON_SYNONYMOUS,7,missense,0.417219,0.013333,NaN,NaN,NaN,NaN,K,N,ENSG00000074800,ENST00000646539,ENO1,CCDS97.1,NaN,4/12,872.0,0.371855,192.0,0.147126,64.0,0.147465,ndomain,NaN,NaN,798,328,tolerated,0.10,benign,0.043,0.998,1,1,0.418,2.907,3.021,699,NaN,NaN,NaN,NaN,2,0,0,0,0,0,1,17,11,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,684.84,3.272140e-56,19.8,11.2,NaN,NaN,51.9226,7.94278,26.8278,5.17973,14.1775,3.46316,19.4358,5.10922,7.68046,2.38594,39.0690,8.53183,5.17637,1.23906,70.5810,11.8499,91.4021,17.2031,61.2727,11.0441,4.56415,0.99526,0.596333,0.100014,656.417,245.501,94.0,35,0,2,10,2,7,151,16,60,1450,NaN,NaN,NaN,7,7,3.083464,23.2
1,8870500,C,T,SNV,0,Coding

In [ ]:
#Chrom\tPos\tRef\tAlt\tType\tLength\tAnnoType\tConsequence\tConsScore\tConsDetail\tGC\tCpG\tmotifECount\tmotifEName\tmotifEHIPos\tmotifEScoreChng\toAA\tnAA\tGeneID\tFeatureID\tGeneName\tCCDS\tIntron\tExon\tcDNApos\trelcDNApos\tCDSpos\trelCDSpos\tprotPos\trelProtPos\tDomain\tDst2Splice\tDst2SplType\tminDistTSS\tminDistTSE\tSIFTcat\tSIFTval\tPolyPhenCat\tPolyPhenVal\tpriPhCons\tmamPhCons\tverPhCons\tpriPhyloP\tmamPhyloP\tverPhyloP\tbStatistic\ttargetScan\tmirSVR-Score\tmirSVR-E\tmirSVR-Aln\tcHmm_E1\tcHmm_E2\tcHmm_E3\tcHmm_E4\tcHmm_E5\tcHmm_E6\tcHmm_E7\tcHmm_E8\tcHmm_E9\tcHmm_E10\tcHmm_E11\tcHmm_E12\tcHmm_E13\tcHmm_E14\tcHmm_E15\tcHmm_E16\tcHmm_E17\tcHmm_E18\tcHmm_E19\tcHmm_E20\tcHmm_E21\tcHmm_E22\tcHmm_E23\tcHmm_E24\tcHmm_E25\tGerpRS\tGerpRSpval\tGerpN\tGerpS\ttOverlapMotifs\tmotifDist\tEncodeH3K4me1-sum\tEncodeH3K4me1-max\tEncodeH3K4me2-sum\tEncodeH3K4me2-max\tEncodeH3K4me3-sum\tEncodeH3K4me3-max\tEncodeH3K9ac-sum\tEncodeH3K9ac-max\tEncodeH3K9me3-sum\tEncodeH3K9me3-max\tEncodeH3K27ac-sum\tEncodeH3K27ac-max\tEncodeH3K27me3-sum\tEncodeH3K27me3-max\tEncodeH3K36me3-sum\tEncodeH3K36me3-max\tEncodeH3K79me2-sum\tEncodeH3K79me2-max\tEncodeH4K20me1-sum\tEncodeH4K20me1-max\tEncodeH2AFZ-sum\tEncodeH2AFZ-max\tEncodeDNase-sum\tEncodeDNase-max\tEncodetotalRNA-sum\tEncodetotalRNA-max\tGrantham\tDist2Mutation\tFreq100bp\tRare100bp\tSngl100bp\tFreq1000bp\tRare1000bp\tSngl1000bp\tFreq10000bp\tRare10000bp\tSngl10000bp\tEnsembleRegulatoryFeature\tdbscSNV-ada_score\tdbscSNV-rf_score\tRemapOverlapTF\tRemapOverlapC\tRawScore\tPHRED\n"

In [5]:
row = "10_000060001_G_A	SNV	0	Intergenic	INTERGENIC	0	intergenic	0.408940397351	0.0233333333333	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	30652	32824	NA	NA	NA	NA	0.197	0.197	0.203	0.110	0.109	0.111	824	NA	NA	NA	NA	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	0.000	1.000	NA	NA	0	0	NA	NA	NA	NA	NA	NA	NA	1.0	2.04	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	419	NA	NA	NA	0	1	9	19	118	NA	NA	0.433834	8.741"
R = row.split()
headd = "key_id	Type	Length	AnnoType	Consequence	ConsScore	ConsDetail	GC	CpG	motifECount	motifEName	motifEHIPos	motifEScoreChng	oAA	nAA	GeneID	FeatureID	GeneName	CCDS	Intron	Exon	cDNApos	relcDNApos	CDSpos	relCDSpos	protPos	relProtPos	Domain	Dst2Splice	Dst2SplType	minDistTSS	minDistTSE	SIFTcat	SIFTval	PolyPhenCat	PolyPhenVal	priPhCons	mamPhCons	verPhCons	priPhyloP	mamPhyloP	verPhyloP	bStatistic	targetScan	mirSVR-Score	mirSVR-E	mirSVR-Aln	cHmmTssA	cHmmTssAFlnk	cHmmTxFlnk	cHmmTx	cHmmTxWk	cHmmEnhG	cHmmEnh	cHmmZnfRpts	cHmmHet	cHmmTssBiv	cHmmBivFlnk	cHmmEnhBiv	cHmmReprPC	cHmmReprPCWk	cHmmQuies	GerpRS	GerpRSpval	GerpN	GerpS	TFBS	TFBSPeaks	TFBSPeaksMax	tOverlapMotifs	motifDist	Segway	EncH3K27Ac	EncH3K4Me1	EncH3K4Me3	EncExp	EncNucleo	EncOCC	EncOCCombPVal	EncOCDNasePVal	EncOCFairePVal	EncOCpolIIPVal	EncOCctcfPVal	EncOCmycPVal	EncOCDNaseSig	EncOCFaireSig	EncOCpolIISig	EncOCctcfSig	EncOCmycSig	Grantham	Dist2Mutation	Freq100bp	Rare100bp	Sngl100bp	Freq1000bp	Rare1000bp	Sngl1000bp	Freq10000bp	Rare10000bp	Sngl10000bp	dbscSNV-ada_score	dbscSNV-rf_score	RawScore	PHRED"
H = headd.split()
print(len(R))
print(len(H))

103
104


# [1] retrieve coordinates using protein API
## in toolbox : proteinAPIcoordinates.py
```python
"""
visit website: https://www.ebi.ac.uk/proteins/api/doc/#!/coordinates/getGenomePositionByAccession
"""
import requests, sys

baseurl = 'https://www.ebi.ac.uk/proteins/api/coordinates/location/'

endpoint = 'UniprotID:#'

#loop thru a column of just uniprotID:# for cysteine

#example of requestURL:  "https://www.ebi.ac.uk/proteins/api/coordinates/location/Q00796:45"

# = None conditional argument
#TODO add argparse for hoffman python sesh

def getCoordinates(request_endpoint, additional_endpoint = None, parameter = None):
    requesturl = baseurl + request_endpoint
    if additional_endpoint:
        requesturl = requesturl + '/' + additional_endpoint

    #flag, parameter is a value evaluates to True
    if parameter:
        if type(parameter) is not str:
            parameter = str(parameter) 
        requesturl = requesturl + '/' + parameter

    print requesturl 

    response = requests.get(requesturl, headers = {"Accept": "application/json"})
    data = response.text 
    headers = response.headers 
    print headers 
    return data 

def main():
    data = getResults(endpoint, 'interaction', 'O43521')
    # in teh main body i should loop thru reactivity data and grab the accession ID and cys position, these should be entered
    # as parameters to the getCoordinates function
    print data

main()
```

In [ ]:
import requests, sys

requestURL = "https://www.ebi.ac.uk/proteins/api/coordinates?accession=P01116"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.text
print(responseBody)

In [ ]:
requestURL = "https://www.ebi.ac.uk/proteins/api/coordinates?accession=P01116-1"

r = requests.get(requestURL, headers={ "Accept" : "text/x-gff"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.text
print(responseBody)

In [ ]:
# checking what genecode v19 transcript matches the UKB sequence for kras
gencode = "/Users/mariapalafox/Box Sync/CODE_DATA/ACCOUNTING/mappingUKB2hg19transcripts/gnomad_exac_ukb_gencodev19Transcript_matched.csv"
code = pd.read_csv(gencode)